In [1]:
from transformers import pipeline,set_seed
from datasets import load_dataset,load_from_disk
from pathlib import Path
import matplotlib.pyplot as plt
import pandas as pd
from datasets import load_metric
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import nltk
from nltk.tokenize import sent_tokenize
import torch

nltk.download('punkt')

c:\Users\dbhav\anaconda3\envs\textS\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dbhav\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
device = "cpu"
print(f"Device: {device}")

Device: cpu


In [3]:
model_ckpt ="google/pegasus-cnn_dailymail"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
dataset = load_from_disk("samsum_dataset")

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [6]:
def convert_to_features(example_batch):
    input_encodings = tokenizer(example_batch["dialogue"], truncation=True, padding="max_length", max_length=1024)
    with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer(example_batch['summary'],max_length=128, truncation=True, padding="max_length")
    return {
        "input_ids":input_encodings['input_ids'],
        "attention_mask":input_encodings['attention_mask'],
        "labels":target_encodings['input_ids']
    }

In [7]:
dataset_samsum_pt = dataset.map(convert_to_features,batched=True)

In [8]:
dataset_samsum_pt['train']

Dataset({
    features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 14732
})

In [9]:
#training

from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [10]:
from transformers import TrainingArguments, Trainer

trainer_args = TrainingArguments(
    output_dir="pegasus_samsum",
    num_train_epochs=1,
    warmup_steps=500,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    weight_decay=0.01,
    logging_steps=10,
    eval_steps=500,
    evaluation_strategy="steps",
    save_steps=1e6,
    gradient_accumulation_steps=16
)

In [11]:
trainer = Trainer(
    model=model,
    args=trainer_args,
    train_dataset=dataset_samsum_pt['test'],
    eval_dataset=dataset_samsum_pt['validation']
)

In [12]:
trainer.train()

c:\Users\dbhav\anaconda3\envs\textS\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  2%|▏         | 1/51 [02:20<1:57:12, 140.65s/it]

OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 4.00 GiB total capacity; 10.24 GiB already allocated; 0 bytes free; 10.25 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF